In [6]:
# Importing libraries

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.utils.data as data
import albumentations as A
from torch.utils.tensorboard import SummaryWriter
from torch.nn.parallel import DistributedDataParallel

In [7]:
# Set random seed for reproducibility

torch.manual_seed(42)

In [9]:
transform_train = A.Compose([
    A.Resize(256, 256),                  # Resize the image to 256x256 pixels
    A.RandomCrop(224, 224),              # Randomly crop a 224x224 patch from the image
    A.HorizontalFlip(p=0.5),             # Randomly flip the image horizontally with a probability of 0.5
    A.Rotate(limit=15),                   # Rotate the image by a random angle within the range [-15, 15] degrees
    A.ColorJitter(brightness=0.2),        # Adjust the brightness of the image by a factor of up to ±0.2
    A.RandomContrast(limit=0.2),          # Adjust the contrast of the image by a factor of up to ±0.2
    A.Normalize(),                        # Normalize the image pixel values
    transforms.ToTensor()                            # Convert the image to a PyTorch tensor
])

In [10]:
transform_validation = A.Compose([
    A.Resize(256, 256),
    A.CenterCrop(224, 224),               # Crop the center region of the image to 224x224 pixels
    A.Normalize(),
    transforms.ToTensor()
])

In [11]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
ROOT_PATH = '/content/drive/MyDrive/VegetableImages'

In [13]:
!pwd

/content


In [14]:
import os

os.chdir(ROOT_PATH)
os.getcwd()

'/content/drive/MyDrive/VegetableImages'

In [19]:
# Defining dataset paths

train_path = "/content/drive/MyDrive/VegetableImages/train/"
validation_path = "/content/drive/MyDrive/VegetableImages/validation"
test_path = "/content/drive/MyDrive/VegetableImages/test"

In [20]:
# Create dataset loaders

train_dataset = datasets.ImageFolder(train_path, transform=transform_train)
validation_dataset = datasets.ImageFolder(validation_path, transform=transform_validation)
test_dataset = datasets.ImageFolder(test_path, transform=transform_validation)


FileNotFoundError: ignored

In [ ]:
# Set batch size and number of workers

batch_size = 64
num_workers = 4

In [ ]:
# Create data loaders

train_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
validation_loader = data.DataLoader(validation_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
test_loader = data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)


In [ ]:
# Define your model architecture

class YourModel(nn.Module):
    def __init__(self):
        super(YourModel, self).__init__()
        # Add your model layers here

    def forward(self, x):
        # Define the forward pass of your model
        return x

In [ ]:
# Initialize your model and move it to a suitable device

model = YourModel()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
# Define loss function and optimizer

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Initialize DistributedDataParallel for parallel training

model = DistributedDataParallel(model)

In [ ]:
# Set up TensorBoard logging

writer = SummaryWriter()

In [ ]:
# Training loop

num_epochs = 10
best_accuracy = 0.0

In [ ]:
# Training loop
num_epochs = 10
best_accuracy = 0.0

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)

    # Calculate average training loss
    train_loss = train_loss / len(train_loader.dataset)

    # Validation
    model.eval()
    total = 0
    correct = 0

    with torch.no_grad():
        for images, labels in validation_loader:
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Calculate validation accuracy
    validation_accuracy = 100 * correct / total

    # Log training and validation performance to TensorBoard
    writer.add_scalar("Loss/train", train)